<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/LLM_GPT2_QA_Finetune_Q_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import transformers
print(transformers.__version__)

4.47.1


Finetuning:

Best

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import os
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token to tokenizer and model
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Example QA dataset
qa_data = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = tokenizer.eos_token_id  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert dataset to Huggingface Dataset object
dataset = Dataset.from_list(qa_data)
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["question", "answer"])

# Define collation function
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


# Data loaders
batch_size = 2
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Define optimizer, criterion, and device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss(ignore_index=tokenizer.eos_token_id)  # Use -100 to ignore irrelevant tokens


def train_model_manual_loss(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=10, save_dir="./gpt2_qa_finetuned"):
    best_val_loss = float("inf")
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Training phase
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # Get the logits directly

            # Shift logits and labels
            shift_logits = logits[..., :-1, :].contiguous()  # Remove the last token
            shift_labels = labels[..., 1:].contiguous()      # Remove the first token

            # Reshape for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))  # (batch_size * seq_len, vocab_size)
            shift_labels = shift_labels.view(-1)  # (batch_size * seq_len)

            # Calculate loss using the criterion
            loss = criterion(shift_logits, shift_labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        print(f"Training Loss: {avg_train_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits

                # Shift logits and labels for validation
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = labels[..., 1:].contiguous()

                # Reshape for loss calculation
                shift_logits = shift_logits.view(-1, shift_logits.size(-1))
                shift_labels = shift_labels.view(-1)

                # Calculate validation loss
                loss = criterion(shift_logits, shift_labels)

                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Validation Loss: {avg_val_loss:.4f}")

        # Save the model if validation loss improves
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            model.save_pretrained(save_dir)
            tokenizer.save_pretrained(save_dir)
            print(f"Saved best model with Validation Loss: {avg_val_loss:.4f}")

# Train the model with manual loss calculation
train_model_manual_loss(model, train_loader, val_loader, optimizer, criterion, device)



# Save the model
model.save_pretrained("./gpt2_qa_finetuned")
tokenizer.save_pretrained("./gpt2_qa_finetuned")

# Test the model
def generate_answer(question, model, tokenizer, device="cuda"):
    model.to(device)  # Move the model to the specified device
    model.eval()  # Set the model to evaluation mode
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)  # Move inputs to the same device
    outputs = model.generate(inputs, max_length=100, num_beams=5, early_stopping=True, pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Load the fine-tuned model and tokenizer for testing
model = GPT2LMHeadModel.from_pretrained("./gpt2_qa_finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_qa_finetuned")

# Test generation
question = "What is the capital of France?"
device = "cuda" if torch.cuda.is_available() else "cpu"
answer = generate_answer(question, model, tokenizer, device=device)
print(answer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

Validating: 100%|██████████| 1/1 [00:00<00:00, 49.13it/s]

Validation Loss: 0.0013


Saved best model with Validation Loss: 0.0013


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Question: What is the capital of France?
Answer: The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France


#Inference: Greedy Search

In [ ]:
import torch
import torch.nn.functional as F
import time

def generate_answer(question, model, tokenizer, max_length=50, device="cuda"):
    """
    Generate an answer using greedy search.
    Args:
        question (str): The input question.
        model (GPT2LMHeadModel): The GPT-2 model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        device (str): Device to run the model on ('cuda' or 'cpu').

    Returns:
        str: The generated answer.
    """
    # Move the model to the correct device
    model.to(device)
    model.eval()  # Set model to evaluation mode

    # Prepare input
    input_text = f"Question: {question}\nAnswer:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Initialize the generated token IDs with the input
    generated_ids = input_ids

    for _ in range(max_length):
        # Forward pass to get logits for the next token
        outputs = model(input_ids=generated_ids)
        logits = outputs.logits

        # Select the token with the highest probability (greedy search)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1).unsqueeze(0)

        # Append the predicted token to the sequence
        generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)

        # Stop if the end-of-sequence (EOS) token is generated
        if next_token_id.item() == tokenizer.eos_token_id:
            break

    # Decode the generated token IDs to a text string
    answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return answer


# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2_qa_finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_qa_finetuned")

# Test the generate_answer function
question = "What is the capital of france"
device = "cuda" if torch.cuda.is_available() else "cpu"
st = time.time()
answer = generate_answer(question, model, tokenizer, device=device)
en = time.time()
print('total time:', en-st)
print(answer)

total time: 0.7245104312896729
Question: What is the capital of france
Answer: The capital of france is Paris. The capital of france is Paris. The capital of france is Paris. The capital of france is Paris. The capital of france is Paris. The capital of france is Paris. The capital


#Inference: Batch Greedy Search

In [ ]:
import torch
import torch.nn.functional as F
import time
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def batch_greedy_search(questions, model, tokenizer, max_length=50, device="cuda"):
    """
    Generate answers for a batch of questions using greedy search.
    Args:
        questions (list): A list of input questions.
        model (GPT2LMHeadModel): The GPT-2 model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        device (str): Device to run the model on ('cuda' or 'cpu').

    Returns:
        list: The generated answers.
    """
    # Move the model to the correct device
    model.to(device)
    model.eval()

    # Prepare inputs with padding
    input_texts = [f"Question: {q}\nAnswer:" for q in questions]
    input_data = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True)
    input_ids = input_data["input_ids"].to(device)
    attention_mask = input_data["attention_mask"].to(device)

    # Add padding token to handle dynamic length expansion
    pad_token_id = tokenizer.pad_token_id
    input_ids = torch.cat([input_ids, torch.full((input_ids.size(0), 1), pad_token_id, dtype=torch.long, device=device)], dim=1)
    attention_mask = torch.cat([attention_mask, torch.zeros((attention_mask.size(0), 1), dtype=torch.long, device=device)], dim=1)

    # Track the length of each input prompt
    input_lengths = attention_mask.sum(dim=1)  # Number of non-padding tokens per input
    initial_input_lengths = input_lengths.clone()

    batch_size = len(questions)

    with torch.no_grad():
        for idx in range(max_length):
            # print(input_ids, attention_mask)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Select the token using input_lengths to avoid padding issues
            next_token_logits = logits[torch.arange(batch_size), input_lengths - 1, :]
            next_token_ids = torch.argmax(next_token_logits, dim=-1).unsqueeze(1)
            # if idx == 0 or 1:
            #     print('b1 next_token_ids:', next_token_ids[0], next_token_ids.shape)
            #     print('b1 next_token_ids:', next_token_ids[0][0].item(), tokenizer.decode(next_token_ids[0][0]))
            #     print('b2 next_token_ids:', next_token_ids[1][0].item(), tokenizer.decode(next_token_ids[1][0]))

            # Stop generating if EOS token is predicted for all sequences
            eos_reached = next_token_ids == tokenizer.eos_token_id
            if eos_reached.all():
                break

            # Append predicted tokens and update attention mask
            input_ids.scatter_(1, input_lengths.unsqueeze(1), next_token_ids)
            attention_mask.scatter_(1, input_lengths.unsqueeze(1), torch.ones_like(next_token_ids))
            input_lengths += 1

            # Expand input_ids and attention_mask for further predictions
            input_ids = torch.cat([input_ids, torch.full((batch_size, 1), pad_token_id, dtype=torch.long, device=device)], dim=1)
            attention_mask = torch.cat([attention_mask, torch.zeros((batch_size, 1), dtype=torch.long, device=device)], dim=1)

    # Extract only the generated part by slicing from input length onwards
    answers = []
    for i in range(batch_size):
        generated_answer_ids = input_ids[i, initial_input_lengths[i]:]
        print('input_ids:', input_ids[i].shape)
        print('generated_answer_ids:', generated_answer_ids)
        eos_idx = (generated_answer_ids == tokenizer.eos_token_id).nonzero(as_tuple=True)[0]
        if eos_idx.numel() > 0:
            generated_answer_ids = generated_answer_ids[:eos_idx[0]]

        answer = tokenizer.decode(generated_answer_ids, skip_special_tokens=True).strip()
        answers.append(answer)

    return answers


# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2_qa_finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_qa_finetuned")

# Sample questions
questions = [
    "What is the capital of France?",
    "Who wrote Hamlet?",
]

device = "cuda" if torch.cuda.is_available() else "cpu"
st = time.time()
answers = batch_greedy_search(questions, model, tokenizer, device=device)
en = time.time()
print('Total time:', en - st)
for q, a in zip(questions, answers):
    print(f"Q: {q}\nA: {a}\n")


input_ids: torch.Size([63])
generated_answer_ids: tensor([  383,  3139,   286,  4881,   318,  6342,    13,   383,  3139,   286,
         4881,   318,  6342,    13,   383,  3139,   286,  4881,   318,  6342,
           13,   383,  3139,   286,  4881,   318,  6342,    13,   383,  3139,
          286,  4881,   318,  6342,    13,   383,  3139,   286,  4881,   318,
         6342,    13,   383,  3139,   286,  4881,   318,  6342,    13,   383,
        50256], device='cuda:0')
input_ids: torch.Size([63])
generated_answer_ids: tensor([ 4502, 32226,  2630,  4345,  1616,    13,  4502, 32226,  2630,  4345,
         1616,    13,  4502, 32226,  2630,  4345,  1616,    13,  4502, 32226,
         2630,  4345,  1616,    13,  4502, 32226,  2630,  4345,  1616,    13,
         4502, 32226,  2630,  4345,  1616,    13,  4502, 32226,  2630,  4345,
         1616,    13,  4502, 32226,  2630,  4345,  1616,    13,  4502, 32226,
        50256, 50256, 50256], device='cuda:0')
Total time: 2.451781988143921
Q: What is

#Inference: Beam Search

In [ ]:
import torch
import torch.nn.functional as F
import time

def generate_answer(question, model, tokenizer, max_length=50, num_beams=5, device="cuda"):
    """
    Generate an answer using beam search for the fine-tuned GPT-2 model.

    Args:
        question (str): The input question.
        model (GPT2LMHeadModel): The fine-tuned model.
        tokenizer (GPT2Tokenizer): The tokenizer.
        max_length (int): Maximum length of the generated text.
        num_beams (int): Number of beams for beam search.
        device (str): The device to run the model on ('cuda' or 'cpu').

    Returns:
        str: The generated answer.
    """
    model.to(device)
    model.eval()

    # Prepare input text
    input_text = f"Question: {question}\nAnswer:"
    inputs = tokenizer(input_text, return_tensors="pt", padding=True)

    # Extract input_ids and attention_mask
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Initialize beams
    beams = [(input_ids, attention_mask, 0.0)]  # List of (sequence, attention_mask, score)
    completed_sequences = []

    # Beam search loop
    for _ in range(max_length):
        new_beams = []

        for seq, mask, score in beams:
            # Stop expanding if sequence ends with EOS token
            if seq[0, -1] == tokenizer.eos_token_id:
                completed_sequences.append((seq, score))  # Append sequence and score
                continue

            # Forward pass
            with torch.no_grad():
                outputs = model(seq, attention_mask=mask)
                logits = outputs.logits[:, -1, :]  # Logits of the last token
                probs = F.log_softmax(logits, dim=-1)  # Convert to log probabilities

            # Get top-k tokens and their log probabilities
            top_k_probs, top_k_tokens = torch.topk(probs, num_beams, dim=-1)

            # Expand each beam
            for prob, token in zip(top_k_probs[0], top_k_tokens[0]):
                new_seq = torch.cat([seq, token.unsqueeze(0).unsqueeze(0)], dim=1)  # Append token
                new_mask = torch.cat([mask, torch.ones((1, 1), device=device)], dim=1)  # Update attention mask
                new_score = score + prob.item()  # Accumulate log probability as float
                new_beams.append((new_seq, new_mask, new_score))

        # Sort new beams by score and keep top-k
        new_beams = sorted(new_beams, key=lambda x: x[2], reverse=True)[:num_beams]
        beams = new_beams

        # Break if all beams end with EOS token
        if all(seq[0, -1] == tokenizer.eos_token_id for seq, _, _ in beams):
            break

    # Add remaining beams to completed sequences
    completed_sequences.extend([(seq, float(score)) for seq, _, score in beams])

    # Select the sequence with the highest score
    best_sequence, _ = max(completed_sequences, key=lambda x: x[1])

    # Decode the tokens to text
    answer = tokenizer.decode(best_sequence[0], skip_special_tokens=True)
    return answer


# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./gpt2_qa_finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2_qa_finetuned")

# Test the generate_answer function
question = "What is the capital of France?"
device = "cuda" if torch.cuda.is_available() else "cpu"
st = time.time()
answer = generate_answer(question, model, tokenizer, max_length=50, num_beams=5, device=device)
en = time.time()
print('total time:', en-st)
print(answer)

total time: 5.485363960266113
Question: What is the capital of France?
Answer: The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The capital of France is Paris. The


Debugging:

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import Dataset
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import os
import random
import numpy as np

# Seed setting function
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility
seed = 50
set_seed(seed)

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add padding token to tokenizer and model
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Example QA dataset
qa_data = [
    {"question": "What is the capital of France?", "answer": "The capital of France is Paris."},
    {"question": "Who wrote '1984'?", "answer": "George Orwell wrote '1984'."},
]

# Preprocess dataset
def preprocess_data(example):
    input_text = f"Question: {example['question']}\nAnswer: {example['answer']}"
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=60)

    # Clone input_ids into labels
    labels = inputs["input_ids"].copy()

    # Mask question tokens and padding tokens in labels
    question_length = len(tokenizer(f"Question: {example['question']}\nAnswer:")["input_ids"]) - 1
    for i in range(len(labels)):
        if i < question_length or labels[i] == tokenizer.pad_token_id:
            labels[i] = tokenizer.eos_token_id  # Ignore question and padding tokens

    inputs["labels"] = labels
    return inputs

# Convert dataset to Huggingface Dataset object
dataset = Dataset.from_list(qa_data)
tokenized_dataset = dataset.map(preprocess_data, remove_columns=["question", "answer"])

# Define collation function
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


# Data loaders
batch_size = 2
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Define optimizer, criterion, and device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = CrossEntropyLoss(ignore_index=tokenizer.eos_token_id)  # Use -100 to ignore irrelevant tokens


def train_model_manual_loss(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=1, save_dir="./gpt2_qa_finetuned"):
    best_val_loss = float("inf")
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")

        # Training phase
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc="Training"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            print('\ninput_ids:', input_ids.shape,'\ntoken ids:',input_ids, '\ndecoded text:', tokenizer.decode(input_ids[0]))
            print('labels:', labels.shape,'\ntoken ids:',labels)

            # Forward pass
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits  # Get the logits directly

            # Shift logits and labels
            shift_logits = logits[..., :-1, :].contiguous()  # Remove the last token
            shift_labels = labels[..., 1:].contiguous()      # Remove the first token
            print('shift_labels:', shift_labels.shape,'\ntoken ids:',shift_labels)

            # Reshape for loss calculation
            shift_logits = shift_logits.view(-1, shift_logits.size(-1))  # (batch_size * seq_len, vocab_size)
            shift_labels = shift_labels.view(-1)  # (batch_size * seq_len)

            # Calculate loss using the criterion
            loss = criterion(shift_logits, shift_labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            break

        avg_train_loss = train_loss / len(train_loader)
        print(f"Training Loss: {avg_train_loss:.4f}")

# Train the model with manual loss calculation
train_model_manual_loss(model, train_loader, val_loader, optimizer, criterion, device)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Epoch 1/1


Training:   0%|          | 0/1 [00:00<?, ?it/s]


input_ids: torch.Size([2, 60]) 
token ids: tensor([[24361,    25,  1867,   318,   262,  3139,   286,  4881,    30,   198,
         33706,    25,   383,  3139,   286,  4881,   318,  6342,    13, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256],
        [24361,    25,  5338,  2630,   705, 28296, 30960,   198, 33706,    25,
          4502, 32226,  2630,   705, 28296,  4458, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256]],
     

Training:   0%|          | 0/1 [00:00<?, ?it/s]

Training Loss: 1.3734


In [ ]:
Inf-1:
Input- Question: What is the capital of France? \nAnswer:

Output- Question: What is the capital of France? \nAnswer: The

Inf-2:
Input- Question: What is the capital of France? \nAnswer: The

Output- Question: What is the capital of France? \nAnswer: The capital

Object `France` not found.


In [ ]:
Input- Question: What is the capital of France? \nAnswer:

